In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
pd.options.display.max_columns = None
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import svm, datasets

In [2]:
df = pd.read_csv('Employee_Attrition.csv')

In [3]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


In [4]:
df.shape

(1470, 35)

In [5]:
objList = df.select_dtypes(include = "object").columns
print (objList)

Index(['Attrition', 'BusinessTravel', 'Department', 'EducationField', 'Gender',
       'JobRole', 'MaritalStatus', 'Over18', 'OverTime'],
      dtype='object')


In [6]:
#Label Encoding for object to numeric conversion
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

for feat in objList:
    df[feat] = le.fit_transform(df[feat].astype(str))

print (df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   Age                       1470 non-null   int64
 1   Attrition                 1470 non-null   int32
 2   BusinessTravel            1470 non-null   int32
 3   DailyRate                 1470 non-null   int64
 4   Department                1470 non-null   int32
 5   DistanceFromHome          1470 non-null   int64
 6   Education                 1470 non-null   int64
 7   EducationField            1470 non-null   int32
 8   EmployeeCount             1470 non-null   int64
 9   EmployeeNumber            1470 non-null   int64
 10  EnvironmentSatisfaction   1470 non-null   int64
 11  Gender                    1470 non-null   int32
 12  HourlyRate                1470 non-null   int64
 13  JobInvolvement            1470 non-null   int64
 14  JobLevel                  1470 non-null 

In [7]:
#separate the other attributes from the predicting attribute
X = df.drop('Attrition',axis=1)
#separte the predicting attribute into Y for model training 
y = df['Attrition']

In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [9]:
from sklearn.decomposition import PCA
pca = PCA(0.95)
#pca=PCA(n_components=15) 

In [10]:
pca.fit(X) 
X_pca=pca.transform(X) 
#let's check the shape of X_pca array
print("shape of X_pca", X_pca.shape)

shape of X_pca (1470, 24)


In [11]:
pca.explained_variance_ratio_

array([0.15117765, 0.06114354, 0.05816788, 0.05464114, 0.05146782,
       0.03853271, 0.03703788, 0.03657881, 0.0359128 , 0.03438039,
       0.03333633, 0.03248468, 0.03232453, 0.0316036 , 0.03111195,
       0.03055598, 0.030281  , 0.02869389, 0.02842834, 0.02746784,
       0.0271386 , 0.02388697, 0.0173601 , 0.0167328 ])

In [12]:
X_recovered = pca.inverse_transform(X_pca)

In [13]:
X_recovered.shape

(1470, 34)

In [14]:
X_recovered

array([[ 0.16957826,  0.57646843,  0.71877358, ...,  0.03297941,
        -0.65201194,  0.09165322],
       [ 1.105674  , -0.90514168, -1.32033994, ...,  0.62192245,
        -0.39541936,  0.76455932],
       [-0.01977497,  0.58368645,  1.41521321, ..., -1.09043453,
        -0.66955694, -1.20925878],
       ...,
       [-1.10255184,  0.58779736, -1.63027207, ..., -0.43243387,
        -0.63685486, -0.28746044],
       [ 1.3816356 , -0.92921305,  0.56906358, ...,  0.5580288 ,
        -0.73181158,  0.84771857],
       [-0.39091705,  0.56877306, -0.40673862, ..., -0.47890233,
        -0.33690535, -0.51107464]])

In [15]:
# importing train_test_split from sklearn
from sklearn.model_selection import train_test_split
# splitting the data
x_train, x_test, y_train, y_test = train_test_split(X_pca, y, test_size = 0.2, random_state = 100)

In [16]:
x_train.shape

(1176, 24)

In [17]:
from sklearn.svm import SVC
svm = SVC()
svm.fit(x_train,y_train)

SVC()

In [20]:
from sklearn.model_selection import GridSearchCV
param_grid = {'C':[0.1,1,10,100], 'gamma':[1,0.1,0.01,0.001], 'kernel':['linear', 'poly', 'rbf', 'sigmoid']}

grid = GridSearchCV(SVC(), param_grid, refit = True, verbose=3)
grid.fit(x_train, y_train)

Fitting 5 folds for each of 64 candidates, totalling 320 fits
[CV 1/5] END ..................C=0.1, gamma=1, kernel=linear; total time=   0.0s
[CV 2/5] END ..................C=0.1, gamma=1, kernel=linear; total time=   0.0s
[CV 3/5] END ..................C=0.1, gamma=1, kernel=linear; total time=   0.0s
[CV 4/5] END ..................C=0.1, gamma=1, kernel=linear; total time=   0.0s
[CV 5/5] END ..................C=0.1, gamma=1, kernel=linear; total time=   0.0s
[CV 1/5] END ....................C=0.1, gamma=1, kernel=poly; total time=   0.1s
[CV 2/5] END ....................C=0.1, gamma=1, kernel=poly; total time=   0.0s
[CV 3/5] END ....................C=0.1, gamma=1, kernel=poly; total time=   0.1s
[CV 4/5] END ....................C=0.1, gamma=1, kernel=poly; total time=   0.0s
[CV 5/5] END ....................C=0.1, gamma=1, kernel=poly; total time=   0.0s
[CV 1/5] END .....................C=0.1, gamma=1, kernel=rbf; total time=   2.6s
[CV 2/5] END .....................C=0.1, gamma=

GridSearchCV(estimator=SVC(),
             param_grid={'C': [0.1, 1, 10, 100], 'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             verbose=3)

In [21]:
grid.best_estimator_.get_params()

{'C': 0.1,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 1,
 'kernel': 'linear',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [23]:
from sklearn.svm import SVC
svm = SVC(C=0.1, gamma = 1, kernel = "linear")
svm.fit(x_train,y_train)

svm.score(x_test,y_test)

1.0

In [22]:
pipe = Pipeline([('classifier' , SVC())])

# Create param grid.

param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.linspace(1, 10, 20),
    'classifier__solver' : ['liblinear']},
    {'classifier' : [GradientBoostingClassifier()],
    'classifier__n_estimators' : list(range(10,101,10)),
     'classifier__learning_rate' : np.linspace(0.1, 10, 50)},
     {'classifier' : [GaussianNB()],'classifier__var_smoothing' :np.linspace(0.0000001, 0.001,4)},
    {'classifier' : [SVC()],
         'classifier__C' : np.linspace(0.1, 10, 20),
    'classifier__kernel' : ['linear', 'poly', 'rbf']}
    

   
]

# Create grid search object

clf = GridSearchCV(pipe, param_grid = param_grid, cv = 3, verbose=True, n_jobs=-1)

# Fit on data

best_clf = clf.fit(x_train, y_train)

Fitting 3 folds for each of 604 candidates, totalling 1812 fits
